# LSTM을 이용한 텍스트 생성
- 데이터 다운로드: 캐글 [New York Times Comments](https://www.kaggle.com/aashita/nyt-comments)
- ArticlesApril2018.csv

## 1. 데이터 Colab으로 업로드

In [ ]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving ArticlesApril2018.csv to ArticlesApril2018.csv


In [ ]:
import pandas as pd

df=pd.read_csv(filename, encoding='latin1')
df.head(2)

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleadersâ Settlement Offer...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"âI understand that they could meet with us, ...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...


## 2. 데이터 전처리

In [ ]:
#너무기니까 ㅋ.ㅋ
print('열의 개수: ', len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [ ]:
# headline 열의 데이터만 사용할것이기때문에, Null 검사
df['headline'].isnull().values.any()

False

In [ ]:
# headline 열의 데이터로 리스트 생성
headline = [text for text in df.headline.values]
headline[:5]

['Former N.F.L. Cheerleadersâ\x80\x99 Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [ ]:
len(headline) 
#1324개나 있다...

1324

In [ ]:
# 노이즈 데이터('Unknown') 제거(널데이터)
headline = [text for text in headline if text != 'Unknown'] #headline이 Unknown이 아닌것만 가져온다
len(headline)
#1214개

1214

In [ ]:
from string import punctuation
punctuation
#!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ 구둣점들 총출동

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
# 구두점 제거와 소문자화를 위한 함수
def preprocessing(s):
    s=s.encode("utf8").decode("ascii", 'ignore')
    return ''.join(c for c in s if c not in punctuation).lower() 

In [ ]:
headlines = [preprocessing(x) for x in headline]
headlines[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
# 단어 집합(vocabulary)을 만들고 크기를 확인
t = Tokenizer()
t.fit_on_texts(headlines)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)
#3494개 #print(t.word_index)하면 너무많음 ㅋㅋ

단어 집합의 크기 : 3494


In [ ]:
type(t.word_index)
#딕셔너리

dict

In [ ]:
sequences = []
for line in headlines: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [ ]:
index_to_word = {}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 1번 단어 :', index_to_word[1]) 
print('빈도수 상위 582번 단어 :', index_to_word[582]) 

빈도수 상위 1번 단어 : the
빈도수 상위 582번 단어 : offer


In [ ]:
max_len = max(len(s) for s in sequences)
print('샘플의 최대 길이 :', max_len) 

샘플의 최대 길이 : 24


In [ ]:
# 전체 샘플의 길이를 24(가장 긴 샘플의 길이)로 패딩
# 'pre' 옵션을 주면 앞을 0으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [ ]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [ ]:
# 레이블 데이터 y에 대해서 원-핫 인코딩을 수행
Y = to_categorical(y, num_classes=vocab_size)

In [ ]:
X.shape, Y.shape

((7803, 23), (7803, 3494))

## 2. 모델 설계 및 학습0.9159

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [ ]:
# 임베딩 벡터는 10차원, 은닉 상태 크기는 128
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1,  
                    name="Embedding_Layer"))
model.add(LSTM(128, name="LSTM_Layer")) 
model.add(Dense(vocab_size, activation='softmax', name="Output_Layer"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 23, 10)            34940     
_________________________________________________________________
LSTM_Layer (LSTM)            (None, 128)               71168     
_________________________________________________________________
Output_Layer (Dense)         (None, 3494)              450726    
Total params: 556,834
Trainable params: 556,834
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X, Y, epochs=200, verbose=2) 

Epoch 1/200
244/244 - 9s - loss: 7.6540 - accuracy: 0.0304
Epoch 2/200
244/244 - 1s - loss: 7.1120 - accuracy: 0.0309
Epoch 3/200
244/244 - 1s - loss: 6.9778 - accuracy: 0.0305
Epoch 4/200
244/244 - 1s - loss: 6.8513 - accuracy: 0.0434
Epoch 5/200
244/244 - 1s - loss: 6.7027 - accuracy: 0.0445
Epoch 6/200
244/244 - 1s - loss: 6.5351 - accuracy: 0.0469
Epoch 7/200
244/244 - 1s - loss: 6.3494 - accuracy: 0.0505
Epoch 8/200
244/244 - 1s - loss: 6.1588 - accuracy: 0.0559
Epoch 9/200
244/244 - 1s - loss: 5.9654 - accuracy: 0.0614
Epoch 10/200
244/244 - 1s - loss: 5.7858 - accuracy: 0.0682
Epoch 11/200
244/244 - 1s - loss: 5.6144 - accuracy: 0.0718
Epoch 12/200
244/244 - 1s - loss: 5.4515 - accuracy: 0.0775
Epoch 13/200
244/244 - 1s - loss: 5.2935 - accuracy: 0.0819
Epoch 14/200
244/244 - 1s - loss: 5.1453 - accuracy: 0.0856
Epoch 15/200
244/244 - 1s - loss: 5.0011 - accuracy: 0.0968
Epoch 16/200
244/244 - 1s - loss: 4.8638 - accuracy: 0.1034
Epoch 17/200
244/244 - 1s - loss: 4.7331 - accura

## 3. 모델 검증

In [ ]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = np.argmax(model.predict(encoded), axis=-1)
          # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장

    sentence = init_word + sentence
    return sentence

In [ ]:
import numpy as np
print(sentence_generation(model, t, 'i', 10))
# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성
print(sentence_generation(model, t, 'how', 10))
# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성
print(sentence_generation(model, t, 'former', 10))

i cant jump ship from facebook yet police in house inspector
how to make a crossword puzzle way to art to australia
former judge to review files seized in searches use say been
